In [1]:
# https://www.dlology.com/blog/how-to-convert-trained-keras-model-to-tensorflow-and-make-prediction/

In [2]:
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

Using TensorFlow backend.


In [1]:
from keras.models import load_model
model = load_model('save/my_model.h5')
print(model.outputs)
# [<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]
print(model.inputs)
# [<tf.Tensor 'conv2d_1_input:0' shape=(?, 28, 28, 1) dtype=float32>]

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


[<tf.Tensor 'dense_2/Softmax:0' shape=(?, 6) dtype=float32>]
[<tf.Tensor 'conv2d_1_input:0' shape=(?, 66, 100, 1) dtype=float32>]


In [2]:
from keras import backend as K
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 39 variables.
INFO:tensorflow:Converted 39 variables to const ops.


In [3]:
# Save to ./model/tf_model.pb
tf.train.write_graph(frozen_graph, "save", "saved_model.pb", as_text=False)

'save/saved_model.pb'

In [4]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model_file('save/my_model.h5') 
tfmodel = converter.convert() 
open ('save/my_model.tflite' , "wb") .write(tfmodel)

INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.


102466580